### Gas Noise Test

In [ ]:
# IPython cell — robust adapter for different gas_noise.py versions

import importlib, inspect
import matplotlib.pyplot as plt
import numpy as np
import gas_noise

# reload in case gas_noise.py was edited on disk
importlib.reload(gas_noise)

def run_and_plot(min_gwei, max_gwei, *,
                 steps=200, seed=None, label=None,
                 edge_width=0.22, sigma_frac=0.05, switch_period=0,
                 interval_min=0.0, interval_max=0.0):
    min_wei = int(min_gwei * 1e9)
    max_wei = int(max_gwei * 1e9)

    # Base kwargs (common to most versions)
    kwargs = dict(
        min_gas=min_wei,
        max_gas=max_wei,
        steps=steps,
        switch_period=switch_period,
        interval_min=interval_min,   # 0 -> no sleep
        interval_max=interval_max,   # 0 -> no sleep
        seed=seed,
        write_toml=False,
        verbose=False,
    )

    # Adapt to function signature
    params = inspect.signature(gas_noise.generate_gas_sequence).parameters

    # Handle edge/sigma names (new anti-normal versions)
    if 'edge_width_frac' in params:
        kwargs['edge_width_frac'] = edge_width
    elif 'edge_width' in params:
        kwargs['edge_width'] = edge_width

    if 'sigma_frac' in params:
        kwargs['sigma_frac'] = sigma_frac

    # If this is an older Beta-U version, fall back to its API
    if 'model' in params and 'beta_alpha' in params:
        # Switch to the beta_u path; remove anti-normal-specific kwargs
        kwargs.update(model='beta_u', beta_alpha=0.5, beta_beta=0.5)
        kwargs.pop('edge_width_frac', None)
        kwargs.pop('edge_width', None)
        kwargs.pop('sigma_frac', None)

    prices, times = gas_noise.generate_gas_sequence(**kwargs)
    prices_gwei = np.array(prices) / 1e9
    plt.plot(times, prices_gwei, marker='o', alpha=0.7, label=label)

# --- demo ---
plt.figure(figsize=(12, 6))
run_and_plot(
    0.01, 1.20,
    steps=1000,
    seed=5,
    label='Base (0.01~1.20 gwei)',
    edge_width=0.22,
    sigma_frac=0.05,
    switch_period=12,
    interval_min=0.0,  # no real sleep in notebooks
    interval_max=0.0
)
plt.axhline(0.62, linestyle='--', label='Target 0.62 gwei')
plt.xlabel('Time (s)'); plt.ylabel('Gas Price (gwei)')
plt.title('Gas Noise Simulation (auto-adapted to gas_noise.py)')
plt.legend(); plt.grid(True); plt.show()


TypeError: generate_gas_sequence() got an unexpected keyword argument 'edge_width_frac'

<Figure size 1200x600 with 0 Axes>